In [1]:
import os
import pandas as pd
import pdpipe as pdp
import spacy
import sys

from IPython.display import display
from os.path import join
from pathlib import Path

project_root = Path('..')
sys.path.append(os.path.abspath(project_root))
from notebooks.utils import init_data_dir  # noqa

from notebooks import pipes  # noqa

init_data_dir(project_root)

preprocess_path = join(project_root, Path('data/preprocess'))

nlp = spacy.load('en_core_web_sm')

df = pd.read_hdf(join(preprocess_path, Path('bawe_df.hdf5')))

# Information for the British Academic Written English Corpus

This notebook is for gathering information about the BAWE dataset. The dataset should already be parsed and stored in `data/preprocess/bawe_df.hdf5`.

In [2]:
df

,author,genre,text
0,1,a,Racism is still a problem within our society t...
1,1,b,Official statistics are those produced by eith...
2,1,c,Since the fourteenth century the practice of m...
3,1,d,Much more reproductive choice is now available...
4,2,a,Victorian notions of women's madness were larg...
...,...,...,...
2572,6998,a,E. Warwick Slinn describes dramatic monologue ...
2573,6998,b,Hugh Blair voices an attack on the practices o...
2574,6998,c,"'The first thing to remember about Donne,' wri..."
2575,6998,d,Susan Wiseman calculated that the latest possi...


In [3]:
resample_splits = True

train_df_path = join(preprocess_path, 'bawe_train_df.hdf5')
valid_df_path = join(preprocess_path, 'bawe_valid_df.hdf5')

train_df_exists = os.path.exists(train_df_path)
valid_df_exists = os.path.exists(valid_df_path)

if not (train_df_exists and valid_df_exists) or resample_splits:
    print('Resampling...')

    train_df = df.sample(frac=0.8).sort_values(by=['author', 'genre'])
    valid_df = df.drop(train_df.index)

    train_df = train_df.reset_index(drop=True)
    valid_df = valid_df.reset_index(drop=True)

    train_df.to_hdf(train_df_path, key='bawe_train_df')
    valid_df.to_hdf(valid_df_path, key='bawe_valid_df')
else:
    train_df = pd.read_hdf(train_df_path)
    valid_df = pd.read_hdf(valid_df_path)

print('Train Set:')
display(train_df)
print('Validation Set:')
display(valid_df)

Resampling...
Train Set:


,author,genre,text
0,1,a,Racism is still a problem within our society t...
1,1,c,Since the fourteenth century the practice of m...
2,2,a,Victorian notions of women's madness were larg...
3,2,b,The development of feminist thought and action...
4,3,b,"On Friday afternoon, the 26 th of November, I ..."
...,...,...,...
2057,6215,e,<heading>INTRODUCTION</heading>Both Macrochloa...
2058,6215,f,<list></list><abstract><heading>Abstract</head...
2059,6998,a,E. Warwick Slinn describes dramatic monologue ...
2060,6998,c,"'The first thing to remember about Donne,' wri..."


Validation Set:


,author,genre,text
0,1,b,Official statistics are those produced by eith...
1,1,d,Much more reproductive choice is now available...
2,3,a,"The publication of The Bell Curve in 1994, the..."
3,3,h,The emergence of the sociological school of et...
4,5,c,For too long the history of the Cold War has b...
...,...,...,...
510,6211,i,This essay is essentially a reaction to an art...
511,6211,j,The Carolingian Renaissance is simultaneously ...
512,6214,c,<heading>Forest Growth Rates</heading><table/>...
513,6998,b,Hugh Blair voices an attack on the practices o...


In [4]:
pipeline = pdp.PdPipeline([pipes.IDText(),
                           pipes.SplitText(nlp, show_loading=True)])

train_df = pipeline(train_df)
valid_df = pipeline(valid_df)

print('Train set:', flush=True)
display(train_df)
print('Validation set:', flush=True)
display(valid_df)

100%|██████████| 515/515 [02:04<00:00,  4.57it/s]

Train set:


sentence
author text_id sentence_position                                                   
1      0       0                  Racism is still a problem within our society t...
               1                  Ethnic minorities are concentrated into certai...
               2                  This can be seen in a survey, carried out in 1...
               3                  In this essay I will look at what racism is, a...
               4                  In this section I will cover three areas that ...
...                                                                             ...
6998   2       175                Yerma never wants the life of a man, yet she i...
               176                She resents her ability to exercise new freedo...
               177                However, Hedda resents her inactivity and depe...
               178                She rejects motherhood and matrimony in favour...
               179                While Hedda aspires to acquire the authority o...

[237872 rows x 1 columns]

Validation set:


sentence
author text_id sentence_position                                                   
1      0       0                  Official statistics are those produced by eith...
               1                  There has been a huge increase in the number o...
               2                  Although they are widely used there are many l...
               3                  In this essay I plan to talk about how the int...
               4                  I will also explore the uses that statistics d...
...                                                                             ...
6998   1       111                As a persuasive text, The Famous Tragedie's cl...
               112                The writer chose to express Parliament's destr...
               113                However, this potential is lost amongst sensat...
               114                While it may have been necessary to produce sh...
               115                Therefore, The Famous Tragedie is heavily flaw...

[59049 rows x 1 columns]

In [5]:
train_df.to_hdf(join(preprocess_path, 'bawe_train_sentences.hdf5'),
                key='bawe_train_sentences')
valid_df.to_hdf(join(preprocess_path, 'bawe_valid_sentences.hdf5'),
                key='bawe_valid_sentences')

In [6]:
237192 / 20

11859.6